# Linear Neural Network - Softmax Regression

In this notebook, we cast the Softmax Regression technique for multiclass classification as a Linear Neural Network (LNN) model. 


In [1]:
import numpy as np
import matplotlib.pyplot as plt


from sklearn.datasets import load_iris

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

import tensorflow as tf

np.random.seed(43)

## Decision Boundary: Using Various Thresholds for a Class Probability

We define a function to show the decision boundaries at various thresholds of a class probability. Using the different estimates for a single class, we generate a contour plot. 

In [2]:
def decision_boundary_class_colored(clf, X):
    
    # Get the min and max value of feature x1
    x1min, x1max = X[:,0].min() - 1, X[:, 0].max() + 1
    
    # Get the min and max value of feature x2
    x2min, x2max = X[:,1].min() - 1, X[:, 1].max() + 1
    
    # Create the mesh grid
    x1s = np.linspace(x1min, x1max, 100)
    x2s = np.linspace(x2min, x2max, 100)
    x1, x2 = np.meshgrid(x1s, x2s)
    
    
    # Create pairs of new points from the grid
    X_new = np.c_[x1.ravel(), x2.ravel()]
    
    
    # Compute the class predictions for all new points        
    y_pred = np.argmax(clf.predict(X_new), axis=-1).reshape(x1.shape)
    
   
    # Generate the contourf plot for the predictions
    plt.contourf(x1, x2, y_pred, cmap=plt.cm.summer, alpha=0.9)

## Multiclass Dataset


We will use the iris dataset, which is a multivariate data set.

This is a famous dataset that contains the sepal and petal length and width of 150 iris flowers of three different species: Iris-Setosa, Iris-Versicolor, and Iris-Virginica

There are 4 features:

- sepal length (cm)
- sepal width (cm)
- petal length (cm)
- petal width (cm)


We will use two features (i.e., petal length, petal width).

In [3]:
iris = load_iris()

X = iris["data"][:, (2, 3)]  # petal length, petal width
y = iris["target"]

label = y

plt.figure(figsize=(12, 6))    
plt.plot(X[y==2, 0], X[y==2, 1], "g^", label="Iris-Virginica")
plt.plot(X[y==1, 0], X[y==1, 1], "bs", label="Iris-Versicolor")
plt.plot(X[y==0, 0], X[y==0, 1], "yo", label="Iris-Setosa")

plt.xlabel("Petal length", fontsize=14)
plt.ylabel("Petal width", fontsize=14)
plt.legend(loc="upper left", fontsize=14)
plt.title("LNN: Multiclass Classification (Softmax Regression)", fontsize=16)
plt.xticks(())
plt.yticks(())

plt.show()

## Create a LNN Model

Using the Keras Dense layer we create a LNN model for Softmax Regression. **In a Dense layer, each neuron is connected to all inputs/neurons of the previous layer**.

The LNN model:
- has 3 neurons because it's a 3-class classification problem.
- uses the softmax activation function.

At first, we need to specify the shape of the input using the Input layer. The Input layer is used to instantiate a Keras tensor, which is a TensorFlow symbolic tensor object.
- We must set the "shape" parameter, which is a tuple (integers) indicating the dimension of the input vectors. In this demo, the input is a 2D vector, that's why we simply put [2, ] in the tuple. In image classification problems, an image input will be represented as a matrix, in which case we will need to specify the size of the matrix. For example, for a 28 x 28 image input matrix, the "shape" tuple will be [28, 28]. Note that we don't need to include the batch size.

Dense layer:
- kernel_initializer: we initialize the weights (also known as the kernel) with zero values. In logistic/softmax regression, initial zero weight values are ok. However, in nonlinear neural networks, such as Multi-layer Perceptron (MLPs), we must use small random values to initialize the weights. For that, we need to use sophisticated initializers, such as glorot or he.
- activation: we use softmax activation

In [4]:
%%time

'''
Delete the TensorFlow graph before creating a new model, otherwise memory overflow will occur.
'''
tf.keras.backend.clear_session()

'''
To reproduce the same result by the model in each iteration, we use fixed seeds for random number generation. 
'''
np.random.seed(42)
tf.random.set_seed(42)


'''
Create a Sequential model composed of a single stack of layers connected sequentially. 
'''
model = tf.keras.models.Sequential(name="LNN_Multiclass_Classifier")
model.add(tf.keras.layers.Input(shape=[2,]))
model.add(tf.keras.layers.Dense(units=3, kernel_initializer="zeros", activation="sigmoid"))

model.summary()

Model: "LNN_Multiclass_Classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 3)                 9         
                                                                 
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________
CPU times: user 52.3 ms, sys: 4.92 ms, total: 57.3 ms
Wall time: 21.8 ms


## Compile & Train the LNN Model

A Keras model automatically uses the **Backpropagation** algorithm to train an ANN. However, we need to specify an optimization algorithm. 

There are various advanced optimizers. For the LNN we will use the **Stochastic Gradient Descent (SGD)** optimizer. It requires us to set a suitable learning rate.  

We will use the mini-batch SGD. Thus we need to specify the batch size as well as the epochs.

In [5]:
'''
Define the optimizer
'''
optimizer = tf.keras.optimizers.SGD(learning_rate=0.3)

'''
Compile the model
'''
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

'''
Set the epochs and batch size
'''
no_of_epochs = 50
size_of_mini_batch = 64

'''
Train the model
'''
history = model.fit(X, y, 
                    batch_size=size_of_mini_batch, 
                    epochs=no_of_epochs,
                    verbose=1)

Epoch 1/50
3/3 [==============================] - 0s 1ms/step - loss: 1.1149 - accuracy: 0.2800
Epoch 2/50
3/3 [==============================] - 0s 782us/step - loss: 1.1647 - accuracy: 0.3533
Epoch 3/50
3/3 [==============================] - 0s 649us/step - loss: 0.9388 - accuracy: 0.3800
Epoch 4/50
3/3 [==============================] - 0s 713us/step - loss: 0.9482 - accuracy: 0.4533
Epoch 5/50
3/3 [==============================] - 0s 756us/step - loss: 1.1131 - accuracy: 0.4533
Epoch 6/50
3/3 [==============================] - 0s 748us/step - loss: 0.8773 - accuracy: 0.7133
Epoch 7/50
3/3 [==============================] - 0s 630us/step - loss: 0.7860 - accuracy: 0.6867
Epoch 8/50
3/3 [==============================] - 0s 714us/step - loss: 0.7325 - accuracy: 0.7867
Epoch 9/50
3/3 [==============================] - 0s 594us/step - loss: 0.7663 - accuracy: 0.6800
Epoch 10/50
3/3 [==============================] - 0s 594us/step - loss: 1.0401 - accuracy: 0.5800
Epoch 11/50
3/3 [====

2024-09-18 20:46:39.843020: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


3/3 [==============================] - 0s 607us/step - loss: 0.4897 - accuracy: 0.7667
Epoch 37/50
3/3 [==============================] - 0s 634us/step - loss: 0.7280 - accuracy: 0.6467
Epoch 38/50
3/3 [==============================] - 0s 673us/step - loss: 0.6640 - accuracy: 0.7600
Epoch 39/50
3/3 [==============================] - 0s 638us/step - loss: 0.4351 - accuracy: 0.9000
Epoch 40/50
3/3 [==============================] - 0s 576us/step - loss: 0.4896 - accuracy: 0.7467
Epoch 41/50
3/3 [==============================] - 0s 692us/step - loss: 0.4233 - accuracy: 0.9333
Epoch 42/50
3/3 [==============================] - 0s 636us/step - loss: 0.4197 - accuracy: 0.8933
Epoch 43/50
3/3 [==============================] - 0s 620us/step - loss: 0.4181 - accuracy: 0.9267
Epoch 44/50
3/3 [==============================] - 0s 624us/step - loss: 0.4320 - accuracy: 0.8733
Epoch 45/50
3/3 [==============================] - 0s 581us/step - loss: 0.6228 - accuracy: 0.6800
Epoch 46/50
3/3 [=====

## Evaluate the LNN Model

In [6]:
numOfEpochs = len(history.history['loss'])
print("Epochs: ", numOfEpochs)

print("\nTest Data: Loss & Accuracy: ", model.evaluate(X, y, verbose=0))

y_predicted = model.predict(X)
y_predicted = np.argmax(y_predicted, axis=-1) # get the label/index of the highest probability class

accuracy = np.mean(y_predicted == y)
print("\nAccuracy: ", accuracy)

print("\nConfusion Matrix:")
print(confusion_matrix(y, y_predicted))

print("\nClassification Report:")
print(classification_report(y, y_predicted))

Epochs:  50

Test Data: Loss & Accuracy:  [0.4187166392803192, 0.8733333349227905]
5/5 [==============================] - 0s 332us/step

Accuracy:  0.8733333333333333

Confusion Matrix:
[[50  0  0]
 [ 0 49  1]
 [ 0 18 32]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        50
           1       0.73      0.98      0.84        50
           2       0.97      0.64      0.77        50

    accuracy                           0.87       150
   macro avg       0.90      0.87      0.87       150
weighted avg       0.90      0.87      0.87       150



## Plot The Decision Boundary

313/313 [==============================] - 0s 179us/step


In [7]:
plt.figure(figsize=(12, 6))    

decision_boundary_class_colored(model, X)

plt.plot(X[y==2, 0], X[y==2, 1], "g^", label="Iris-Virginica")
plt.plot(X[y==1, 0], X[y==1, 1], "bs", label="Iris-Versicolor")
plt.plot(X[y==0, 0], X[y==0, 1], "yo", label="Iris-Setosa")

plt.xlabel("Petal length", fontsize=14)
plt.ylabel("Petal width", fontsize=14)
plt.legend(loc="upper left", fontsize=14)
plt.title("LNN: Binary Classification (Logistic Regression)", fontsize=16)
plt.xticks(())
plt.yticks(())
plt.show()